In [1]:
import pandas as pd
import numpy as np
import os.path
from datetime import datetime, date
from datetime import timedelta

In [2]:
path = './artem-pyanykh-cmc-prac-task3-seed17/out/input'

file_list = []

for f in os.listdir(path):
    file_list.append(os.path.join(path, f))
    
file_list.sort()


In [3]:
def read_scv_shop_info(file_list, i):
    
    invent = pd.read_csv(file_list[i]) # inventory
    sell = pd.read_csv(file_list[i+1]) # sellings
    supp = pd.read_csv(file_list[i+2]) # arrivals
     
    return invent, sell, supp

In [4]:
def better_date(invent, sell, supp):
    
    supp['date'] = pd.to_datetime(supp['date'])
    invent['date'] = pd.to_datetime(invent['date'])
    sell['date'] = pd.to_datetime(sell['date'])
    
    return invent, sell, supp

In [5]:
def proper_sell(sell):
    
    sell['actual'] = pd.Series(x[6:8] for x in sell['sku_num'])
    sell = sell.drop(['sku_num'], axis = 1)
    sell = sell.groupby('date')['actual'].value_counts()
    
    new_sell = pd.DataFrame()

    new_sell['actual'] = sell.index.get_level_values(1)
    new_sell['date'] = sell.index.get_level_values(0)
    new_sell['count'] = sell.reset_index(drop=True)
    sell = new_sell
    
    # разворачиваем таблицу sell в широкую таблицу, что бы появились колонки для яблок и ручек, 
    # так же убираем из индекса дату в отдельный столбец
    sell = sell.pivot_table(columns='actual', index='date', values='count').reset_index()
    
    return sell

In [6]:
#print(invent.head(20))
#print(supp.head(20))
#print(sell.head(100))

In [7]:
def apple_pen_raw_table(sell, supply):
    # объединяем таблицы закупок и продаж по дате используем метод left join
    sell_supply = sell.merge(supp.reset_index(), right_on='date', left_on='date', how='left')

    # так как у нас закпки два раза в месяц то заполняем 
    # все пустые значения предудущим "forward fill"
    sell_supply.fillna(method='ffill', inplace =True)

    return sell_supply

In [8]:
def differential_apple_table(sell_supply):
    
    # выделяем нужные данные по яблокам
    apple = sell_supply[['date', 'ap', 'apple', 'index']]
    # группируем по индексу закупок
    apple = apple.groupby(['index', 'apple']).sum()
    # убираем индекс
    apple.reset_index(inplace=True)
    # считаем разницу для каждой закупки, с продажами к тому моменту времени
    apple['diff'] = apple['apple'] - apple['ap']

    # объединяем таблицу яблок с таблицей закупок, что бы можно было вытащить дату
    q = supp.reset_index()[['index', 'date']].merge(apple, on='index')
    q['month'] = q['date'].dt.month
    q['year'] = q['date'].dt.year
    #  у нас две закупки в месяц, поэтому грпируем по году и месяцу и складываем разницу
    apple_diff = q[['month', 'year', 'diff']].groupby(['year','month']).sum()

    return apple_diff

In [9]:
def inventory_date_split(invent):
    
    invent['month'] = invent['date'].dt.month
    invent['year'] = invent['date'].dt.year
    
    return invent

In [10]:
def stolen_apples(invent, apple_diff):
    
    # на складе делаем тоже самое, выводим в индекс год и месяц
    apple_invent = invent[['month', 'year', 'apple']].groupby(['year','month']).sum()
    
    # объединяем таблицу яблок на складе и таблицу с разницей в закупке - продаже
    apple_vals = apple_invent.merge(apple_diff, on=['year', 'month'])
    # смотрим сколько в первый день было на складе
    zero_day = apple_vals['apple'].values[0]
    # считаем разницу следующего с предыдущим, что бы посмотреть сколько появлялось на складе за месяц
    # заполняем начальное значение тес, что бы заполмнили  раньше 
    apple_vals['apple'] = apple_vals['apple'].diff(1).fillna(zero_day)
    
    # считаем разницу
    stolen_apples = apple_vals['diff'] - apple_vals['apple'] 
    
    return stolen_apples

In [11]:
#делаем идентино для ручек

In [12]:
def differential_pen_table(sell_supply):
    
    # выделяем нужные данные по ручкам
    pen = sell_supply[['date', 'pe', 'pen', 'index']]
    # группируем по индексу закупок
    pen = pen.groupby(['index', 'pen']).sum()
    # убираем индекс
    pen.reset_index(inplace=True)
    # считаем разницу для каждой закупки, с продажами к тому моменту времени
    pen['diff'] = pen['pen'] - pen['pe']
    
    # объединяем таблицу ручек с таблицей закупок, что бы можно было вытащить дату
    k = supp.reset_index()[['index', 'date']].merge(pen, on='index')
    k['month'] = k['date'].dt.month
    k['year'] = k['date'].dt.year
    #  у нас две закпки в месяц, поэтому грпируем по году и месяцу и складываем разницу
    pen_diff = k[['month', 'year', 'diff']].groupby(['year','month']).sum()

    return pen_diff

In [13]:
def stolen_pen(invent, pen_diff):

    # на складе делаем тоже самое, выводим в индекс год и месяц
    pen_invent = invent[['month', 'year', 'pen']].groupby(['year','month']).sum()

    # объединяем таблицу ручек на складе и таблицу с разницей в закупке - продаже
    pen_vals = pen_invent.merge(pen_diff, on=['year', 'month'])
    # смотрим сколько в первый день было на складе
    zero_day = pen_vals['pen'].values[0]
    # считаем разницу следующего с предыдущим, что бы посмотреть сколько появлялось на складе за месяц
    # заполняем начальное значение так, что бы заполмнили  раньше 
    pen_vals['pen'] = pen_vals['pen'].diff(1).fillna(zero_day)
    
    # считаем разницу
    stolen_pen = pen_vals['diff'] - pen_vals['pen']
    
    return stolen_pen

In [185]:
def count_daily_invent(invent, sell, supp):

    sell_invent = sell.merge(invent, on = 'date', how = 'left')
    sell_invent_supp = sell_invent.merge(supp, on = 'date',suffixes=('_invent', '_supply'), how = 'left')
    sell_invent_supp.fillna(0, inplace = True)
    #print(sell_invent_supp.head(100))

    daily_invent = pd.DataFrame()
    
    daily_invent['apple'] = sell_invent_supp['apple_supply'].cumsum() - sell_invent_supp['ap'].cumsum()
    daily_invent['pen'] = sell_invent_supp['pen_supply'].cumsum() - sell_invent_supp['pe'].cumsum()
    daily_invent.index = sell_invent_supp['date']
 
    return daily_invent

In [41]:
def sold_stolen(stolen_apples_table, stolen_pen_table, sell):
    
    sell = sell.rename(columns = {'actual' : 'year', 'ap' : 'apple_sold', 'pe' : 'pen_sold'})
    sell['year'] = sell['date'].dt.year
    sell = sell.groupby('year').sum()
    
    stolen_apples_table.name = 'apple_stolen'
    stolen_pen_table.name = 'pen_stolen'
    stolen_apples_table = stolen_apples_table.groupby('year').sum()
    
    stolen_pen_table = stolen_pen_table.groupby('year').sum()
    sell_stolen_ap = sell.merge(stolen_apples_table, on = 'year', how = 'inner')
    sell_stolen_ap_pe = sell_stolen_ap.merge(stolen_pen_table, on = 'year', how = 'inner')
    
    return sell_stolen_ap_pe

In [159]:
def stolen(stolen_apples_table, stolen_pen_table):
    
    date = pd.DataFrame()
    
    date['year'] = stolen_apples_table.index.get_level_values(0)
    date['month'] = stolen_apples_table.index.get_level_values(1)
    date['day'] = 1
    date = pd.to_datetime(date)
    
    st_ap = stolen_apples_table.reset_index(drop = True)
    st_pe = stolen_pen_table.reset_index(drop = True)
    
    stolen = {'apple' : st_ap, 'pen' : st_pe}
    stolen_product = pd.DataFrame(stolen)
    stolen_product[['apple', 'pen']] = stolen_product[['apple', 'pen']].astype(int)
    stolen_product.index = date
    stolen_product.index.name = 'date'
    
    return stolen_product                                                                                       

In [96]:
def print_shop_number(i):
    print('--------------------')
    print( 'Shop', int((i+3)/3))
    print('--------------------')    

In [97]:
invent, sell, supp = read_scv_shop_info(file_list, 0)
invent, sell, supp = better_date(invent, sell, supp)
sell = proper_sell(sell)

In [186]:
pd.set_option('display.max_rows', 100)

for i in range(0, len(file_list), 3):

    invent, sell, supp = read_scv_shop_info(file_list, i)
    invent, sell, supp = better_date(invent, sell, supp)

    sell = proper_sell(sell)
    
    daily_invent = count_daily_invent(invent, sell, supp)   #ex1
    sell_supply = apple_pen_raw_table(sell, supp)
    
    invent = inventory_date_split(invent)
    
    apple_diff = differential_apple_table(sell_supply)    
    stolen_apples_table = stolen_apples(invent, apple_diff)
    
    pen_diff = differential_pen_table(sell_supply) 
    stolen_pen_table = stolen_pen(invent, pen_diff)
    
    stolen_product = stolen(stolen_apples_table, stolen_pen_table)   # ex2
    sell_stolen_ap_pe = sold_stolen(stolen_apples_table, stolen_pen_table, sell)  #ex3
    
    print_shop_number(i)  
    print('\nЗадание 1\n\n', daily_invent)
    print('\nЗадание 2\n\n', stolen_product)
    print('\nЗадание 3\n\n', sell_stolen_ap_pe)
    #print(stolen_pen_table, stolen_apples_table)

<class 'dict'>
--------------------
Shop 1
--------------------

Задание 1

                 apple       pen
date                           
2006-01-01    33271.0    2574.0
2006-01-02    31409.0    2431.0
2006-01-03    29529.0    2260.0
2006-01-04    27732.0    2107.0
2006-01-05    25790.0    1974.0
...               ...       ...
2015-12-27  1577550.0  124041.0
2015-12-28  1575753.0  123881.0
2015-12-29  1573913.0  123742.0
2015-12-30  1572093.0  123602.0
2015-12-31  1570209.0  123478.0

[3652 rows x 2 columns]

Задание 2

             apple  pen
date                  
2006-01-01     10   11
2006-02-01      6    6
2006-03-01      7    6
2006-04-01      6   14
2006-05-01      8    1
...           ...  ...
2015-08-01      5    7
2015-09-01      5    7
2015-10-01      8    7
2015-11-01      6    5
2015-12-01      7    7

[120 rows x 2 columns]

Задание 3

       apple_sold  pen_sold  apple_stolen  pen_stolen
year                                                
2006      681341     52555 

<class 'dict'>
--------------------
Shop 7
--------------------

Задание 1

                apple      pen
date                         
2006-01-01    2914.0    323.0
2006-01-02    2817.0    313.0
2006-01-03    2721.0    305.0
2006-01-04    2616.0    293.0
2006-01-05    2527.0    287.0
...              ...      ...
2015-12-27  381498.0  43602.0
2015-12-28  381415.0  43592.0
2015-12-29  381304.0  43584.0
2015-12-30  381214.0  43576.0
2015-12-31  381112.0  43562.0

[3652 rows x 2 columns]

Задание 2

             apple  pen
date                  
2006-01-01      5    1
2006-02-01      3    0
2006-03-01      2    3
2006-04-01      1    2
2006-05-01      2    2
...           ...  ...
2015-08-01      3    2
2015-09-01      3    2
2015-10-01      4    2
2015-11-01      5    1
2015-12-01      1    3

[120 rows x 2 columns]

Задание 3

       apple_sold  pen_sold  apple_stolen  pen_stolen
year                                                
2006       33793      3588          25.0        23.0
